In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/ppo-LunarLander-v2/resolve/main/replay.mp4" type="video/mp4"></video>

In [4]:
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [5]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
    # Take a random action
    action = env.action_space.sample()
    print("Action taken:", action)

    # Do this action in the environment and get
    # next_state, reward, terminated, truncated and info
    observation, reward, terminated, truncated, info = env.step(action)

    # If the game is terminated (in our case we land, crashed) or truncated (timeout)
    if terminated or truncated:
        # Reset the environment
        print("Environment is reset")
        observation, info = env.reset()

env.close()

Action taken: 0
Action taken: 1
Action taken: 3
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 0
Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 3
Action taken: 3
Action taken: 1
Action taken: 0
Action taken: 2
Action taken: 2
Action taken: 0


In [6]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())  # Get a random observation

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [ 1.372883   -1.0576133  -2.3163476   0.39687398 -1.9826541  -4.6948724
  0.7413707   0.32081044]


In [7]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample())  # Take a random action


 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 0


In [5]:
# Create the environment
env = gym.make("LunarLander-v2", render_mode="human")

In [7]:
# SOLUTION
# We added some parameters to accelerate the training
model = PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
model.learn(total_timesteps=1000000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99.2     |
|    ep_rew_mean     | -139     |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 21       |
|    total_timesteps | 1024     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 99.5         |
|    ep_rew_mean          | -141         |
| time/                   |              |
|    fps                  | 47           |
|    iterations           | 2            |
|    time_elapsed         | 43           |
|    total_timesteps      | 2048         |
| train/                  |              |
|    approx_kl            | 0.0010876261 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.37        |
|    explained_variance   | -0.00357     |
|    learning_r

KeyboardInterrupt: 

In [14]:
model_name = "ppo-LunarLander-v2"
model.save(model_name)

In [13]:
eval_env = Monitor(gym.make("LunarLander-v2", render_mode="human"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=1, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")
env.close()

mean_reward=215.86 +/- 0.0
